In [2]:
import json

data = {"name": "Dulana", "role": "AI researcher"}

# Convert to JSON string
json_str = json.dumps(data)
print(json_str)

# Convert back to Python dict
loaded = json.loads(json_str)
print(loaded["name"])


{"name": "Dulana", "role": "AI researcher"}
Dulana


###  01 List

In [3]:
from typing import List

names: List[str] = ["Alice", "Bob", "Charlie"]


In [6]:
names[0]

'Alice'

### 02 Dict

In [4]:
from typing import Dict

student: Dict[str, int] = {"math": 95, "science": 88}


In [5]:
student["math"]

95

### 03 Any

In [7]:
from typing import Any

value: Any = 42
value = "hello"


In [8]:
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage


In [9]:
from langchain_core.messages import HumanMessage

msg = HumanMessage(content="Explain LangGraph.")
print(msg)


content='Explain LangGraph.' additional_kwargs={} response_metadata={}


In [ ]:
# LLM sees it as:
{"role": "user", "content": "Explain LangGraph."}

{'role': 'user', 'content': 'Explain LangGraph.'}

In [11]:
from langchain_core.messages import AIMessage

msg = AIMessage(content="LangGraph is a framework for agent workflows.")
print(msg)


content='LangGraph is a framework for agent workflows.' additional_kwargs={} response_metadata={}


In [15]:
# LLM sees it as:
{"role": "assistant", "content": "..."}

{'role': 'assistant', 'content': '...'}

In [16]:
from langchain_core.messages import ToolMessage

tool_output = ToolMessage(
    content="Search results: AI agents are evolving...",
    tool_call_id="search_1"
)

print(tool_output)


content='Search results: AI agents are evolving...' tool_call_id='search_1'


🧱 BaseMessage

The parent class for all message types.

You rarely use it directly — but useful for type hints.

In [17]:
from langchain_core.messages import BaseMessage

def print_message(msg: BaseMessage):
    print(msg.type, msg.content)

print_message(HumanMessage(content="Hi"))
print_message(AIMessage(content="Hello"))


human Hi
ai Hello


🔍 Where these messages are used?
Example: History list

In [18]:
history: List[BaseMessage] = [
    HumanMessage(content="Explain AI agents"),
    AIMessage(content="AI agents are..."),
    HumanMessage(content="Explain reflection agents")
]


In [21]:
from langchain_community.tools import TavilySearchResults
from dotenv import load_dotenv

load_dotenv()
search = TavilySearchResults(max_results=3)

results = search.invoke({"query": "Latest research on Agentic AI"})

for i in results:
    print(i)


{'url': 'https://mitsloan.mit.edu/ideas-made-to-matter/4-new-studies-about-agentic-ai-mit-initiative-digital-economy', 'content': '# 4 new studies about agentic AI from the MIT Initiative on the Digital Economy\n\nBy\n\nJun 17, 2025\n\n## Why It Matters\n\nNew research ranges from how AI agents negotiate to how “personality pairing” can optimize human-AI collaboration.\n\nOver time, artificial intelligence tools are being given more autonomy. Beyond serving as human assistants, they are being programmed to be agents themselves — negotiating contracts, making decisions, exploring legal arguments, and so on. [...] “We are already well into the Agentic Age [of AI],” Aral said. “Companies are developing and deploying autonomous, multimodal AI agents in a vast array of tasks. But our understanding of how to work with AI agents to maximize productivity and performance, as well as the societal implications of this dramatic turn toward agentic AI, is nascent, if not nonexistent. [...] for a ca

hasattr() is a built-in Python function that checks whether an object has a specific attribute.

In [22]:
hasattr(object, "attribute_name")


False

In [23]:
class Person:
    def __init__(self):
        self.name = "Alice"

p = Person()

print(hasattr(p, "name"))      # True
print(hasattr(p, "age"))       # False


True
False


In [36]:
from langchain_core.messages import AIMessage
from pydantic import BaseModel,Field

In [37]:
class AIMessage(BaseModel):
    content: str
    tool_calls: List[dict] = None


In [39]:
msg = AIMessage(
    content="",
    tool_calls=[{"name": "search", "args": {"query": "AI"}}]
)
hasattr(msg, "tool_calls")  # True


True

In [ ]:
import json
from typing import List, Dict, Any
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage
from langchain_community.tools import TavilySearchResults
from dotenv import load_dotenv

load_dotenv()

tavily_tool = TavilySearchResults(max_results=2)

def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    last_ai_message: AIMessage = state[-1]
    if not hasattr(last_ai_message,"tool_calls") or not last_ai_message.tool_calls:
        return []
    
    # hasattr(Object,attribute) --> function

    tool_messages = []

    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuection","ReviseAnswer"]:
            call_id = tool_call["id"]
            search_queries = tool_call["args"].get("search_queries",[])
            """
                ✅ 2. .get("search_queries", [])
                .get() is a dictionary method:
                dict.get(key, default_value)
                Meaning:
                        If the dictionary contains key, return its value.
                        Otherwise, return the default_value.
            """
            query_results = {}
            for query in search_queries:
                result = tavily_tool.invoke(query)
                query_results[query] = result

            tool_messages.append(
                ToolMessage(
                    content = json.dumps(query_results),
                    tool_call_id=call_id
                )
            )

    return tool_messages


"""ToolMessage

    ToolMessage(
        content="...",          # tool response as string
        tool_call_id="12345"    # must match the tool call ID from AIMessage
    )

    {
        "tool_calls": [
            {"id": "1", "name": "search", "args": {...}},
            {"id": "2", "name": "calculator", "args": {...}}
        ] 
    }

"""

# Example usage
test_state = [
    HumanMessage(
        content="Write about how small business can leverage AI to grow"
    ),
    AIMessage(
        content="", 
        tool_calls=[
            {
                "name": "AnswerQuection",
                "args": {
                    'answer': '', 
                    'search_queries': [
                            'capital of sri lanka,give single word answer', 
                            'who is anura kumara dissanayake,give single word answer', 
                            'best engineering university in sri lanka,give single word answer'
                    ], 
                    'reflection': {
                        'missing': '', 
                        'superfluous': ''
                    }
                },
                "id": "call_KpYHichFFEmLitHFvFhKy1Ra",
            }
        ],
    )
]

results = execute_tools(test_state)

#print("Raw results:", results)
if results:
    parsed_content = json.loads(results[0].content)
    #print("Parsed content:", parsed_content)
    #print("\n")


""" 
    {
  "capital of sri lanka,give single word answer": [
    {
      "url": "https://en.wikipedia.org/wiki/Capital_of_Sri_Lanka",
      "content": "... extracted text ..."
    },
    {
      "url": "https://en.wikipedia.org/wiki/Sri_Lanka",
      "content": "... extracted text ..."
    }
  ],

  "who is anura kumara dissanayake,give single word answer": [
    {
      "url": "https://en.wikipedia.org/wiki/Anura_Kumara_Dissanayake",
      "content": "..."
    },
    {
      "url": "https://www.britannica.com/biography/Anura-Kumara-Dissanayake",
      "content": "..."
    }
  ],

  "best engineering university in sri lanka,give single word answer": [
    {
      "url": "https://edurank.org/engineering/lk/",
      "content": "..."
    },
    {
      "url": "https://www.scimagoir.com/...rankings",
      "content": "..."
    }
  ]
}

"""


Parsed content: {'capital of sri lanka,give single word answer': [{'url': 'https://en.wikipedia.org/wiki/Capital_of_Sri_Lanka', 'content': 'Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# Capital of Sri Lanka\n\nThe current legislative capital of Sri Lanka is Sri Jayawardenepura Kotte and the executive and judicial capital is Colombo. Over the course of the island\'s history, the national capital has been in several locations other than Sri Jayawardenepura Kotte.\n\n| Part of a series on the | [...] | Royal Residences |  Tambapaṇṇī  Upatissagāma  Vijithapura  Anuradhapura  Sigiriya  Rohana  Polonnaruwa  Dambadeniya  Yapahuwa  Kurunegala  Gampola  Dedigama  Sri Jayawardenepura Kotte  Rayigama")  Sitawaka  Kandy |\n| Related dynasties |  Kalinga "Kalinga (historical region)")  Vanga  Shakya  Vedda  Pandyan  Mauryan  Madurai Nayak |\n| Other |  Sinhapura  Sinhabahu  Titles")  Crown Jewels")  Royal Standards")  Pretenders")  Abhisheka  Vijayabā Kollaya | [...] | Epochs |  Mahāvaṃsa  V

' \n    {\n  "capital of sri lanka,give single word answer": [\n    {\n      "url": "https://en.wikipedia.org/wiki/Capital_of_Sri_Lanka",\n      "content": "... extracted text ..."\n    },\n    {\n      "url": "https://en.wikipedia.org/wiki/Sri_Lanka",\n      "content": "... extracted text ..."\n    }\n  ],\n\n  "who is anura kumara dissanayake,give single word answer": [\n    {\n      "url": "https://en.wikipedia.org/wiki/Anura_Kumara_Dissanayake",\n      "content": "..."\n    },\n    {\n      "url": "https://www.britannica.com/biography/Anura-Kumara-Dissanayake",\n      "content": "..."\n    }\n  ],\n\n  "best engineering university in sri lanka,give single word answer": [\n    {\n      "url": "https://edurank.org/engineering/lk/",\n      "content": "..."\n    },\n    {\n      "url": "https://www.scimagoir.com/...rankings",\n      "content": "..."\n    }\n  ]\n}\n\n'

In [ ]:
from google.generativeai import list_models
import google.generativeai as genai

genai.configure(api_key="your key")

models = list_models()
for m in models:
    print(m.name, m.supported_generation_methods)

c:\Users\Asus\.conda\envs\agent2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-2.5-flash ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite-preview-02-05 ['generateContent', 'countTokens', 'createCachedContent'